In [170]:
import numpy as np

def generate_fruit_data(n_samples):
    np.random.seed(42)
    
    # Яблоки - более четкое разделение характеристик
    apples = np.array([
        [np.random.randint(70, 80),    # размер
         np.random.randint(150, 200),  # вес
         np.random.randint(1, 3)]      # цвет (1-красный, 2-зеленый)
        for _ in range(n_samples)])
    
    # Апельсины
    oranges = np.array([
        [np.random.randint(65, 70),    # размер
         np.random.randint(140, 150),  # вес
         3]                            # цвет (3-оранжевый)
        for _ in range(n_samples)])
    
    # Груши
    pears = np.array([
        [np.random.randint(80, 90),    # размер
         np.random.randint(190, 250),  # вес
         2]                            # цвет (2-зеленый)
        for _ in range(n_samples)])
    
    X = np.vstack((apples, oranges, pears))
    y = np.array([0] * n_samples + [1] * n_samples + [2] * n_samples)
    
    # Нормализация данных до целых чисел в диапазоне [0, 255]
    X = ((X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) * 255).astype(np.uint16)
    
    return X, y
    
    return X, y

def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x // np.sum(exp_x, axis=1, keepdims=True)

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.randint(-2147483648, 2147483647, (input_size, hidden_size), dtype=np.int32)
        self.W2 = np.random.randint(-2147483648, 2147483647, (hidden_size, output_size), dtype=np.int32)
        
    def forward(self, X):
        self.hidden = relu(np.dot(X, self.W1))
        self.output = np.dot(self.hidden, self.W2)
        return self.output
    
    def train(self, X, y, learning_rate=1, epochs=100):
        for epoch in range(epochs):
            hidden = relu(np.dot(X, self.W1))
            output = np.dot(hidden, self.W2)
            
            output_error = output.copy()
            output_error[range(len(y)), y] -= 1
            
            W2_update = np.dot(hidden.T, output_error)
            W1_update = np.dot(X.T, np.dot(output_error, self.W2.T) * (hidden > 0))
            
            self.W2 -= np.round(learning_rate * W2_update).astype(dtype=np.int32)
            self.W1 -= np.round(learning_rate * W1_update).astype(dtype=np.int32)
    
    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


X, y = generate_fruit_data(1000)


indices = np.random.permutation(len(X))
train_size = int(0.8 * len(X))

train_indices, test_indices = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]

nn = SimpleNeuralNetwork(input_size=3, hidden_size=12, output_size=3)

nn.train(X_train, y_train, learning_rate=2, epochs=1)
predictions = nn.predict(X_test)
accuracy = np.mean(predictions == y_test)

# Вывод весов для последующей реализации в Verilog
print("\nВеса первого слоя:")
print(nn.W1)
print("\nВеса второго слоя:")
print(nn.W2)
print(f"Точность на тестовой выборке: {accuracy * 100:.2f}%")


Веса первого слоя:
[[ -454996554  -573452968 -1258151196  1397665377  1201370261  -985793058
   1079622585 -1519680263   -84539594  1621863634 -1139021143  1868562276]
 [ 1897459154  1687441477  -611096783  -330970726  -582756023 -1656522774
   -759461662  1226736485  1266271558 -1773343744  -358899020  1666988427]
 [  150726393  1382555343  -119026178   114321864   132076829  1751999911
    932063341  1074323116 -1387960849 -1874907674  1362391673   173851644]]

Веса второго слоя:
[[  306560016  -542153421  -222775760]
 [  -17152357  1608425002  2084805836]
 [-1232288061   651085842 -1028851947]
 [-1815176023 -2107120125 -2010415300]
 [-1118736236  -535648356  1563704829]
 [ -916298121  -974737234  -854610678]
 [ 2128076619  -830314614  1793928731]
 [ 2014264660  1876990793 -1796313916]
 [ -475506502 -2146160349  1807429237]
 [  131882255   518500520  1103926397]
 [-1964676085  1216619382  1356540283]
 [-1403932081 -1364877099   491420526]]
Точность на тестовой выборке: 37.50%
